In [1]:
import numpy as np
import pandas as pd
import math 
from datetime import datetime, timedelta

# Random Variates
import random

random.seed(2025)  ## MUST PUT RANDOM SEED LINE IN EACH CELL PRODUCING A RANDOM VARIABLE :( 

# Uniform(a,b)
    # random.uniform(a,b)

# Exponential(lambda), mean 1/lambda
    # random.expovariate(lambd)

# Normal dist mean mu, std.dev sigma
    # random.gauss(mu, sigma)
    
# driver inter = 3
# driver shift = (5,8)
# rider inter = 30
# rider patience = 5
# locations = (0,20)
# trip time= mu = (euc_dist/20)*60  # minutes --> uniform(0.8*mu, 1.2*mu)

# Random Variable Generators

In [2]:
# generate values from random variables

# Driver shift start times
def generate_driver_interarr():
    driver_interarr = round(random.expovariate(3)*60, 2)
    return driver_interarr

def generate_driver_shift():
    driver_shift = round(random.uniform(5,8)*60, 2)
    return driver_shift

def generate_rider_interarr():
    rider_interarr = round(random.expovariate(30)*60, 2)
    return rider_interarr

def generate_rider_patience():
    rider_patience = round(random.expovariate(5)*60, 2)
    return rider_patience

def generate_loc():
    point = round(random.uniform(0,20), 2)
    return point

def generate_trip_time(trip_distance):
    mu = (trip_distance/20)*60 # average time in minutes
    actual_trip_time = round(random.uniform(0.8*mu, 1.2*mu), 2)
    return actual_trip_time

# Functions for events and changes to system state

In [3]:
# How to add next event to event calendar 
def add_next_event(interarr_time, event_type, event_info):
    """ Add next event to event queue (event calendar) then sort queue by time """
#     global now 
    event_time = now + interarr_time # get time of next event (based on interarr times of event)
    event_queue.append([event_time, event_type, event_info]) # add list of [time of next event, event type, and event information] to event_queue 
    event_queue.sort(key=lambda x: x[0])  # Sort event queue by time for executing simulation


In [4]:
# dealing with arrival of driver (aka driver starts shift)
def handle_arrival_of_driver(event_time): 
    """ Everything to do when a driver starts their shift """
    # generate data
    driver_id = len(all_drivers) + 1  # driver ID
    driver_start_x = generate_loc()   # starting x-coord
    driver_start_y = generate_loc()   # starting y-coord
    driver_shift_dur = generate_driver_shift()   # shift duration
    time_until_next_driver = generate_driver_interarr() # time til next driver starts
       
    # put data into dictionary for that driver
    driver= {'driver_ID': driver_id,
             'loc_x':driver_start_x, 
             'loc_y': driver_start_y,
             'shift_end': event_time + driver_shift_dur, # time event starts + shift length
             'next_available': event_time}  # make available at start of shift
    
    # add driver arrival to event queue
#     for_eq = [event_time, 'driver_arrival', driver]
#     event_queue.append(for_eq)
    
    # add driver ID to list of all drivers
    all_drivers.append(driver_id)
    
    # add driver (dictionary) to list of available drivers 
    available_drivers.append(driver)
    
    # make next driver arrival (time, event type, event_info)
    add_next_event(interarr_time = time_until_next_driver, 
                   event_type = 'driver_arrival', 
                   event_info = {})
    # put driver shift end in calendar
    add_next_event(interarr_time = driver_shift_dur, 
                   event_type = 'driver_shift_end', 
                   event_info = {})
    
#     return driver

In [5]:
# dealing with a rider arrival (aka request in system)
def handle_arrival_of_rider(event_time):
    """ Everything to do when a rider requests a ride """
    # generate data
    rider_id = len(all_riders) + 1
    rider_pickup_x = generate_loc()
    rider_pickup_y = generate_loc()
    rider_dropoff_x = generate_loc()
    rider_dropoff_y = generate_loc()
    rider_patience_time = generate_rider_patience()
    time_until_next_rider = generate_rider_interarr()    
    trip_dist = round(math.sqrt((rider_dropoff_x - rider_pickup_x)**2 + (rider_dropoff_y - rider_pickup_y)**2),2)
    trip_length =  generate_trip_time(trip_dist)
    
    # put data into dictionary for that rider
    rider = {'rider_ID': rider_id, 
             
             'rider_request_time': event_time,
             'patience_time': rider_patience_time,
             'abandonment_time': event_time + rider_patience_time,
             'pickup_time': 0,
             'dropoff_time': 0,

             'pickup_x':rider_pickup_x,
             'pickup_y': rider_pickup_y,
             
             'dropoff_x':rider_dropoff_x,
             'dropoff_y': rider_dropoff_y,
             
             'trip_dist': trip_dist,
             'trip_time': trip_length,
             
             'completion_status': 'waiting'}
    
    # add rider arrival to event queue
#     for_eq = [event_time, 'rider_arrival', rider]
#     event_queue.append(for_eq)
    
    # add rider ID number to list of all riders
#     all_riders.append(rider_id)
    all_riders.append([event_time, rider_id])
    
    # add rider dictionary to end of list waiting riders
    riders_waiting.append(rider)
    
    # try to match rider to driver immediately 
    handle_driver_rider_assignment()
    
    # create next rider arrival 
    add_next_event(interarr_time = time_until_next_rider,
                  event_type = 'rider_arrival',
                  event_info = {})    

#     return time_until_next_rider


# produces rider information dictionary 

In [6]:
# dealing with assigning riders and drivers
def handle_driver_rider_assignment():  # dont need event time because does not happen at specific intervals
    """ How to assign a driver to a rider """
    global chosen_rider, chosen_driver
    distances = []
    wait_til_avail = []
    if len(riders_waiting) > 0:  # there is a rider waiting to be picked up 
        
        chosen_rider = riders_waiting[0]  # designate first waiting rider as chosen_rider

        # check if there are any available_drivers 
        if len(available_drivers) > 0:
            # check euclidean distance between rider and all drivers
            for i in available_drivers:
                # find closest driver
                euc_dist = round(math.sqrt(
                    (chosen_rider['pickup_x']-i['loc_x'])**2 +
                    (chosen_rider['pickup_y']-i['loc_y'])**2),2)
                val = [i['driver_ID'], euc_dist]
                distances.append(val) # make list of IDs and Euclidean distances
                min_ans = min(distances, key=lambda x: x[1])  # find min Euclidean distance
                # print(min_ans, 'closest driver is #', min_ans[0], 'who is', min_ans[1], 'miles away')
                
                
            # remove closest driver from available_driver list, add to busy_driver list
            for i in available_drivers:
                if i['driver_ID'] == min_ans[0]:
                    chosen_driver = i
            available_drivers.remove(chosen_driver)
            busy_drivers.append(chosen_driver)
            
# #           # remove closest driver from available_driver list, add to busy_driver list
#             for i in range(len(available_drivers)):
#                 if available_drivers[i].get('driver_ID') == min_ans[0]:  # find closest driver in list of avail drivers
#                     chosen_driver = available_drivers[i]   # get driver dictionary, name him chosen_driver
#                     available_drivers.remove(chosen_driver)  # take closest driver out of list of avail drivers
#                     busy_drivers.append(chosen_driver)  # add driver to list of busy drivers 
                    
            # update lists 
            riders_waiting.pop(0) # remove chosen rider from list of waiting riders b/c has been assigned a driver
            new_pair = [chosen_rider, chosen_driver]  # make a list of chosen_rider and chosen_driver
            rider_driver_assigned_pairs.append(new_pair)
            
            #update rider status as picked_up
            chosen_rider['completion_status'] = 'assigned'
            
            # generate pick-up time (euc dist current loc to rider)
            time_until_pickup = generate_trip_time(min_ans[1]) # how long for driver to get to rider
            chosen_rider['pickup_time'] = now + time_until_pickup  # adding pickup time to rider dictionary
            
            # create next event 
            add_next_event(interarr_time = time_until_pickup,
                  event_type = 'pickup',
                  event_info = {})
            
        # otherwise if no drivers are currently available...
        elif len(available_drivers) == 0: # there are no currently available drivers
            # get times all busy drivers will next be available
            for i in busy_drivers: 
                next_time = i['next_available'] # get all next_available times
                val = [i['driver_ID'], next_time ] # make small list of ID and next available time   
                wait_til_avail.append(val)  # add list to a list
                min_wait_time = min(wait_til_avail, key = lambda x: x[1])  # find minimum next_avail times from list of lists

                if i.get('driver_ID') == min_wait_time[0]:  # find driver dictionary of next avail driver in list of busy drivers
                    chosen_driver = i 
             
            # check if next dropoff time is < patience time
            if chosen_driver['next_available'] < chosen_rider['abandonment_time']:  # if driver available before abandonment time
                # calculate euclidean distance from drop-off loc to rider loc 
                dist_to_get = round(math.sqrt(
                                        (chosen_driver['loc_x'] - chosen_rider['pickup_x'])**2 +
                                        (chosen_driver['loc_y'] - chosen_rider['pickup_y'])**2),2)
                # calculate time until pickup
                time_until_pickup = generate_trip_time(dist_to_get)
                chosen_rider['pickup_time'] = now + time_until_pickup
                
                # update rider status as assigned to a driver
                chosen_rider['completion_status'] = 'assigned'
                
                # update lists
                riders_waiting.pop(0) # remove chosen rider from list of waiting riders 
                new_pair = [chosen_rider, chosen_driver] # get pair of assigned rider-drivers
                rider_driver_assigned_pairs.append(new_pair) # add pair to final list of pairs
                
                # next event is rider gets picked up
                add_next_event(interarr_time = time_until_pickup,
                  event_type = 'pickup',
                  event_info = {})
            
            # otherwise, can't assign soon enough --> rider abanadons     
            else: 
                riders_waiting.pop(0) # get chosen_rider out of waiting_rider list
                riders_abandoned.append(chosen_rider) # add rider info to list of abandonded riders
#     return chosen_rider, chosen_driver


# update driver next_available time status ===================================================================

In [7]:
# dealing with pickup time 
def handle_pickup(event_time):
#     global chosen_rider, chosen_driver
    # update driver location to dropoff (x,y)
    # do this now so that busy drivers have next set of coords so can still assign to next rider if must wait for next avail driver b/c all drivers are busy 
    chosen_driver['loc_x'] = chosen_rider['dropoff_x']
    chosen_driver['loc_y'] = chosen_rider['dropoff_y']

    # generate dropoff time 
    dropoff_time = chosen_rider['pickup_time'] + chosen_rider['trip_time']
    chosen_rider['dropoff_time'] = dropoff_time
    
    # add next event 'dropoff'
    add_next_event(interarr_time = chosen_rider['trip_time'],  # should be at dropoff time found above
                  event_type = 'dropoff',
                  event_info = {}) 
    
#     return dropoff_time, chosen_rider

In [8]:
# dealing with dropoff 
def handle_dropoff(event_time):
#     global chosen_rider, chosen_driver
    # if driver shift ends before official dropoff time but are already assigned, make shift end the new dropoff time
    for i in busy_drivers:
        if i['shift_end'] < chosen_rider['dropoff_time']: # < event_time? ===============================================
            i['shift_end'] = chosen_rider['dropoff_time']  # make shift end the same as dropoff time
        
        # take driver out of busy_driver list, put in finished drivers list
        busy_drivers.remove(i)
        finished_drivers.append(i)
        
        # rider already taken from list of waiting_riders --> just add to list of finished riders
        riders_finished.append(i)

        
    else: # driver's shift ends after dropoff and before assignment 
    
        # make next avaialble time
        chosen_driver['next_available'] = event_time

        # add rider to list of riders_finished
        riders_finished.append(chosen_rider)
    
        # if other drivers available next rider will choose from them and won't have to see which busy drivers will be next available
        if len(available_drivers) > 0:
            available_drivers.append(chosen_driver)  # add driver to list of available drivers

        elif len(available_drivers) == 0: # all drivers busy -- have to look at list of busy drivers to find next
            handle_driver_rider_assignment()  # go through driver-rider assignment loop again 
            
#     return finished_drivers

In [9]:
# handle driver shift ending
def handle_driver_shift_end(event_time):  # shift end of a driver that is not currently busy
    """ What to do when (available) driver's shift ends """
    for i in available_drivers:
        if i['shift_end'] <= event_time:
            available_drivers.remove(i)
            finished_drivers.append(i)
#     return i, finished_drivers

# Testing simulation

In [10]:
# lists of events to keep track of 
# random.seed(2025)


event_queue = []

all_drivers = []
available_drivers = []
busy_drivers = []
finished_drivers = []

all_riders = []
riders_waiting = []
riders_finished = []
riders_abandoned = []

rider_driver_assigned_pairs = []

def test_sim(end_time):
    """ Run simulation and see what happens """
    global now 
    now = 0
    
    # make first driver and rider arrival to kickstart the program
    add_next_event(interarr_time = round(random.expovariate(3)*60, 2), 
                   event_type = 'driver_arrival',
                   event_info = {})
    add_next_event(interarr_time = round(random.expovariate(30)*60, 2), 
                   event_type = 'rider_arrival',
                   event_info = {})

    print(f"Starting simulation until {end_time}")
    while event_queue and event_queue[0][0]  < end_time:
#         print(f"Processing event at {event_queue[0][0]} (End Time: {end_time})")
        event_time, event_type, event_info = event_queue.pop(0)
        now = event_time
        print(f"Current time: {now}, Event type: {event_type}")
#         print(f"Processing event at {now} ")

        if event_type == 'driver_arrival':
            handle_arrival_of_driver(event_time)
            
        elif event_type == 'rider_arrival': 
            handle_arrival_of_rider(event_time)  
            
        elif event_type == 'pickup': 
            handle_pickup(event_time)
            
        elif event_type == 'dropoff': 
            handle_dropoff(event_time)
            
        elif event_type == 'driver_shift_end': 
            handle_driver_shift_end(event_time)
            
    return all_riders, all_drivers

# not updating "event time" so it will keep producing events without stopping 

In [11]:
test_sim(end_time = 40)

Starting simulation until 40
Current time: 2.08, Event type: rider_arrival


NameError: name 'chosen_driver' is not defined